In [ ]:
import torch
from transformers import AutoTokenizer, Qwen3MoeConfig, Qwen3MoeForCausalLM
from transformers.utils import logging
from accelerate import init_empty_weights, infer_auto_device_map

# Optional: Helps debugging
logging.set_verbosity_info()

# Model path
# huihui-ai/Huihui-MoE-5B-A1.7B-abliterated
model_path = "Qwen/Qwen3-30B-A3B"

# Load config
config = Qwen3MoeConfig.from_pretrained(model_path)
# Initialize empty model (no weights yet) to calculate device map
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Step 2: Initialize model with empty weights
with init_empty_weights():
    model = Qwen3MoeForCausalLM(config)

# Step 3: Infer device map for GPU 0 and 1 only
# device_map = infer_auto_device_map(
#     model,
#     max_memory={
#         0: "48GiB",
#         1: "48GiB",
#     },
#     no_split_module_classes=["QwenBlock"],  # prevent splitting inside transformer blocks
#     dtype=torch.bfloat16,
# )

# Step 4: Load actual weights with correct map
model = Qwen3MoeForCausalLM.from_pretrained(
    model_path,
    config=config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)
model.eval()


In [3]:
print(model.hf_device_map)


{'model.embed_tokens': 0, 'model.layers.0': 0, 'model.layers.1': 0, 'model.layers.2': 0, 'model.layers.3': 0, 'model.layers.4': 0, 'model.layers.5': 0, 'model.layers.6': 0, 'model.layers.7': 0, 'model.layers.8': 0, 'model.layers.9': 0, 'model.layers.10': 0, 'model.layers.11': 0, 'model.layers.12': 1, 'model.layers.13': 1, 'model.layers.14': 1, 'model.layers.15': 1, 'model.layers.16': 1, 'model.layers.17': 1, 'model.layers.18': 1, 'model.layers.19': 1, 'model.layers.20': 1, 'model.layers.21': 1, 'model.layers.22': 1, 'model.layers.23': 1, 'model.layers.24': 1, 'model.layers.25': 2, 'model.layers.26': 2, 'model.layers.27': 2, 'model.layers.28': 2, 'model.layers.29': 2, 'model.layers.30': 2, 'model.layers.31': 2, 'model.layers.32': 2, 'model.layers.33': 2, 'model.layers.34': 2, 'model.layers.35': 2, 'model.layers.36': 2, 'model.layers.37': 2, 'model.layers.38': 3, 'model.layers.39': 3, 'model.layers.40': 3, 'model.layers.41': 3, 'model.layers.42': 3, 'model.layers.43': 3, 'model.layers.44

In [11]:
if "inputs" in globals():
    del inputs

first_device = model.hf_device_map.get("transformer.wte", "cuda:0")
device = torch.device(first_device)

prompt = 'Zentropa has much in common with The Third Man, another noir-like film set among the rubble of postwar Europe. Like TTM, there is much inventive camera work. There is an innocent American who gets emotionally involved with a woman he doesn\'t really understand, and whose naivety is all the more striking in contrast with the natives.<br /><br />But I\'d have to say that The Third Man has a more well-crafted storyline. Zentropa is a bit disjointed in this respect. Perhaps this is intentional: it is presented as a dream/nightmare, and making it too coherent would spoil the effect. <br /><br />This movie is unrelentingly grim--"noir" in more than one sense; one never sees the sun shine. Grim, but intriguing, and frightening.' +' Is the prior text positive or negative? Give us a rating out of 10'
inputs = tokenizer(prompt, return_tensors="pt")
inputs = {k: v.to(device) for k, v in inputs.items()}

# Move inputs to first device used by the model
# first_device = model.hf_device_map['transformer.wte']  # Or another known input layer
# inputs = {k: v.to(first_device) for k, v in inputs.items()}

# Run inference
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=500,
        use_probabilistic_routing=True,
        prob_routing_temp=0.1,  # Avoid too low values, or it can crash
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Forward Qwen3MoeForCasualLM
Forward Qwen3MoeForCasualLM
Forward Qwen3MoeForCasualLM
Forward Qwen3MoeForCasualLM
Forward Qwen3MoeForCasualLM
Forward Qwen3MoeForCasualLM
Forward Qwen3MoeForCasualLM
Forward Qwen3MoeForCasualLM
Forward Qwen3MoeForCasualLM
Forward Qwen3MoeForCasualLM
Forward Qwen3MoeForCasualLM
Forward Qwen3MoeForCasualLM
Forward Qwen3MoeForCasualLM
Forward Qwen3MoeForCasualLM
Forward Qwen3MoeForCasualLM
Forward Qwen3MoeForCasualLM
Forward Qwen3MoeForCasualLM
Forward Qwen3MoeForCasualLM
Forward Qwen3MoeForCasualLM
Forward Qwen3MoeForCasualLM
Forward Qwen3MoeForCasualLM
Forward Qwen3MoeForCasualLM
Forward Qwen3MoeForCasualLM
Forward Qwen3MoeForCasualLM
Forward Qwen3MoeForCasualLM
Forward Qwen3MoeForCasualLM
Forward Qwen3MoeForCasualLM
Forward Qwen3MoeForCasualLM
Forward Qwen3MoeForCasualLM
Forward Qwen3MoeForCasualLM
Forward Qwen3MoeForCasualLM
Forward Qwen3MoeForCasualLM
Forward Qwen3MoeForCasualLM
Forward Qwen3MoeForCasualLM
Forward Qwen3MoeForCasualLM
Forward Qwen3MoeForC